## Tratamiendo de los datos de SONDA

In [1]:

import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import math

from helper_fn import preprocess_features, preprocess_new_features


ModuleNotFoundError: No module named 'tensorflow'

## Preprocesamiento de los datos

Primero, cargamos los datos desde un archivo CSV

In [ ]:
oportunidades = pd.read_csv("./script/oportunidades2.csv")

La siguiente función toma el archivo ``oportunidades.csv`` y selecciona los atributos utiles para este analisis

Buscamos los dos sets de filial crm y filial no crm

In [42]:
thisyear = pd.to_datetime('today').year
year_str = '1/1/'+str(thisyear)
thisyear = pd.to_datetime(year_str)
#ts = thisyear - pd.DateOffset(years=1)
ts = thisyear - pd.DateOffset(years=1)


#oportunidades.loc[oportunidades.index > thisyear].to_csv('consulta.csv')
filial_crm, filial_nocrm = preprocess_features(oportunidades)
n = len(filial_crm.columns)

filial_crm.drop(['cmtx_unidaddenegocioname', 'Duracion_dias', 'Estado_Perdido'],axis=1, inplace=True)


## Sets de prueba y de entrenamiento
Dividimos la muestra en prueba y entrenamiento, dejando de entrenamiento los datos del 2018. Primero trabajaremos solo con las filiales que son parte del crm, para las cuales tenemos datos de clasificación sensibles.

In [43]:

thisyear = pd.to_datetime('today').year
year_str = '1/1/'+str(thisyear)
thisyear = pd.to_datetime(year_str)
#ts = thisyear - pd.DateOffset(years=1)
ts = thisyear - pd.DateOffset(years=1)


filial_crm = filial_crm.loc[filial_crm.index <= thisyear]
filial_crm.drop(['opportunityid'],axis=1, inplace=True)

to_test_features = filial_crm.loc[:, filial_crm.columns != 'Estado_Ganado'].loc[filial_crm.index > ts]
to_test_target = filial_crm['Estado_Ganado'].loc[filial_crm.index > ts]
to_train_features = filial_crm.loc[:, filial_crm.columns != 'Estado_Ganado'].loc[filial_crm.index <= ts]
to_train_target = filial_crm['Estado_Ganado'].loc[filial_crm.index <= ts]

train = to_train_features
target = to_train_target

features_test = to_test_features
target_test = to_test_target



montototal = filial_crm[['totalamount_base', 'Estado_Ganado']]
montototal = montototal.loc[montototal['Estado_Ganado'] == 1]
montototal = montototal.loc[montototal.index >= ts]
montototal = montototal.loc[montototal.index < thisyear]
filial_crm.drop(['totalamount_base'],axis=1, inplace=True)
montototal.drop(['Estado_Ganado'],axis=1, inplace=True)


## Crear el modelo


In [45]:


def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    dataset = dataset.shuffle(10).repeat().batch(batch_size)
    return dataset




def eval_input_fn(features, labels, batch_size):
    """An input function for evaluation or prediction"""
    features=dict(features)
    if labels is None:
        inputs = features
    else:
        inputs = (features, labels)

    dataset = tf.data.Dataset.from_tensor_slices(inputs)
    
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    return dataset



In [46]:
feature_columns = []

for key in train.keys():
    feature_columns.append(tf.feature_column.numeric_column(key=key))


In [58]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[1 ,],
    n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/gw/mpmkzqcs0wq9b37znq_nz9f00000gn/T/tmpu5d1ktbv', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1057bd710>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


# Train the model

In [59]:
batch_size = 100
train_steps = 400
 
classifier.train(input_fn = lambda: train_input_fn(train, target, batch_size), steps = train_steps)
eval_result = classifier.evaluate(input_fn = lambda: eval_input_fn(features_test, target_test,batch_size))
print(eval_result)



INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/gw/mpmkzqcs0wq9b37znq_nz9f00000gn/T/tmpu5d1ktbv/model.ckpt.
INFO:tensorflow:loss = 167837220000.0, step = 1
INFO:tensorflow:global_step/sec: 365.435
INFO:tensorflow:loss = 85981340.0, step = 101 (0.275 sec)
INFO:tensorflow:global_step/sec: 588.494
INFO:tensorflow:loss = 53.415142, step = 201 (0.170 sec)
INFO:tensorflow:global_step/sec: 614.202
INFO:tensorflow:loss = 78.26722, step = 301 (0.163 sec)
INFO:tensorflow:Saving checkpoints for 400 into /var/folders/gw/mpmkzqcs0wq9b37znq_nz9f00000gn/T/tmpu5d1ktbv/model.ckpt.
INFO:tensorflow:Loss for final step: 226397.83.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-26-18:39:55
I

In [60]:
accuracy_score = classifier.evaluate(input_fn=lambda:eval_input_fn(features_test,labels=target_test,batch_size=batch_size))['accuracy']
print('\nTest Accuracy: {0:f}%\n'.format(accuracy_score*100))


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-26-18:40:02
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/gw/mpmkzqcs0wq9b37znq_nz9f00000gn/T/tmpu5d1ktbv/model.ckpt-400
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-07-26-18:40:03
INFO:tensorflow:Saving dict for global step 400: accuracy = 0.6376812, accuracy_baseline = 0.6376812, auc = 0.5, auc_precision_recall = 0.8188406, average_loss = 0.65719557, global_step = 400, label/mean = 0.6376812, loss = 60.46199, precision = 0.6376812, prediction/mean = 0.6708721, recall = 1.0

Test Accuracy: 63.768119%



# Predicciones

Ahora vemos como se comporta en la predicción de cierre de oportunidad

In [55]:
predictions = classifier.predict(
    input_fn=lambda:eval_input_fn(to_test_features,labels=to_test_target, batch_size=batch_size))

template = ('\nPrediction is {} with ({:.1f}%) probability')

val = []
val1 = []
val2 = []
val3 = []
opt_id = 0
for pred_dict in predictions:
    opt_id +=1
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]
    val1.append(opt_id)
    val2.append(class_id)
    val3.append(probability)
    
    print(template.format(class_id, 100 * probability))

print("Predictions stored in array val")


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/gw/mpmkzqcs0wq9b37znq_nz9f00000gn/T/tmpl3p86cmv/model.ckpt-400
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

Prediction is 1 with (58.6%) probability

Prediction is 1 with (58.6%) probability

Prediction is 1 with (58.6%) probability

Prediction is 1 with (58.6%) probability

Prediction is 1 with (58.6%) probability

Prediction is 1 with (58.6%) probability

Prediction is 1 with (58.6%) probability

Prediction is 1 with (58.6%) probability

Prediction is 1 with (58.6%) probability

Prediction is 1 with (58.6%) probability

Prediction is 1 with (58.6%) probability

Prediction is 1 with (58.6%) probability

Prediction is 1 with (58.6%) probability

Prediction is 1 with (58.6%) probability

Prediction is 1 with (58.6%) probability

Prediction is 1 with (58.6%) probability

Prediction is 1 

In [ ]:
df = pd.DataFrame({'opt_id': val1, 'class_id': val2, 'prob':val3})
df = df.set_index('opt_id')
df.to_csv('results.csv')
#df.loc[df['class_id'] == 1]

In [ ]:
def get_opt_prob_by_id(id_num):
    df = pd.read_csv('results.csv')
    df = df.set_index('opt_id')
    
    ret = df.loc[df.index == id_num].as_matrix()

    return ret[0]

get_class, get_prob = get_opt_prob_by_id(53)

In [ ]:
get_class

In [ ]:
feature_spec = tf.feature_column.make_parse_example_spec(feature_columns)
serving_input_receiver_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)
export_dir = classifier.export_savedmodel('export', serving_input_receiver_fn)
print(export_dir)

In [ ]:

filial_nocrm_chile = filial_nocrm[filial_nocrm['cmtx_filialname'] == 'Sonda Chile']
filial_nocrm_chile['totalamount_base'].reset_index



In [ ]:
from scipy import stats
import numpy as np
x = np.random.random(10)
y = np.random.random(10)
slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)